In [1]:
import csv
from jobspy import scrape_jobs
import pandas as pd


In [2]:
# jobs = scrape_jobs(
#     site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
#     search_term="software engineer",
#     location="Dallas, TX",
#     results_wanted=20,
#     hours_old=72, # (only Linkedin/Indeed is hour specific, others round up to days old)
#     country_indeed='USA',  # only needed for indeed / glassdoor
#     # linkedin_fetch_description=True # get full description and direct job url for linkedin (slower)
# )
# print(f"Found {len(jobs)} jobs")
# print(jobs.head())
# jobs.to_csv("jobs.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False) # to_xlsx

In [3]:
# df = pd.read_csv('jobs.csv')

In [4]:
# df.shape

In [5]:
# df.columns

In [6]:
# df.head(3).T

In [7]:
# filtered_jobs = jobs.loc[:, (jobs.isna().sum() / jobs.shape[0]) < 0.25]

In [8]:
# filtered_jobs.shape

In [9]:
import json
import pandas as pd
import openai
import os
from loguru import logger

import psycopg2
from pgvector.psycopg2 import register_vector
import numpy as np
from psycopg2.extras import execute_values
import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
def get_embeddings(text):
    if isinstance(text, str):  # Check if text is a string
        response = openai.Embedding.create(
            model="text-embedding-ada-002",
            input=text.replace("\n", " ")
        )
        embedding = response['data'][0]['embedding']
        return embedding
    else:
        return None  # Or an empty list or array, depending on your preference


In [11]:
# #Helper function: get embeddings for a text
# def get_embeddings(text):
#    response = openai.Embedding.create(
#        model="text-embedding-ada-002",
#        input = text.replace("\n"," ")
#    )
#    embedding = response['data'][0]['embedding']
#    return embedding

In [19]:
def etl():
    jobs = scrape_jobs(
    site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
    search_term="software engineer",
    location="San Francisco, CA",
    results_wanted=20,
    hours_old=72, # (only Linkedin/Indeed is hour specific, others round up to days old)
    country_indeed='USA',  # only needed for indeed / glassdoor
    # linkedin_fetch_description=True # get full description and direct job url for linkedin (slower)
)
    logger.info(f"{jobs.shape}")
    
    # Transform
    jobs['embeddings'] = jobs['description'].apply(lambda x: get_embeddings(x) if isinstance(x, str) else None)

    # Load
    connection = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="mypass",
        host="localhost",  # or the database server IP address
        port="5432"        # default port for PostgreSQL
    )
    
    cur = connection.cursor()
    #install pgvector
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
    connection.commit()
    
    
    register_vector(connection)
    
    # Create table to store embeddings and metadata
    table_create_command = """
    CREATE TABLE embeddings (
                id bigserial primary key, 
                site text,
                job_url text,
                title text,
                company text,
                location text,
                date_posted text,
                interval text, 
                min_amount double precision,
                max_amount double precision,
                currency text,
                is_remote text,
                description text,
                company_url text,
                company_logo text,
                embedding vector(1536)
                );
                """
    cur.execute(table_create_command)
    cur.close()
    connection.commit()
    
    #Batch insert embeddings and metadata from dataframe into PostgreSQL database
    register_vector(connection)
    cur = connection.cursor()
    # Prepare the list of tuples to insert
    data_list = [
    (
        row['site'],
        row['job_url'],
        row['title'],
        row['company'],
        row['location'],
        row['date_posted'],
        row['interval'],
        row['min_amount'],
        row['max_amount'],
        row['currency'],
        row['is_remote'],
        row['description'],
        row['company_url'],
        row['company_logo'],
        np.array(row['embeddings']).flatten()  # Flatten embeddings to ensure 1D
    )
    for index, row in jobs.iterrows()
]

    # Use execute_values to perform batch insertion
    execute_values(
        cur,
        """
        INSERT INTO embeddings (site, job_url, title, company, location, date_posted, interval, min_amount, max_amount, currency, is_remote, description, company_url, company_logo, embedding)
        VALUES %s
        """,
        data_list
    )
    
    # commit after we insert all embeddings
    connection.commit()

In [21]:
etl()

2024-11-11 16:39:50,728 - INFO - JobSpy:Indeed - search page: 1 / 1
2024-11-11 16:39:50,729 - INFO - JobSpy:LinkedIn - search page: 1 / 2
2024-11-11 16:40:56,405 - ERROR - JobSpy:LinkedIn - LinkedIn: HTTPSConnectionPool(host='www.linkedin.com', port=443): Max retries exceeded with url: /jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=software+engineer&location=San+Francisco%2C+CA&distance=50&pageNum=0&start=0&f_TPR=r259200 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (read timeout=10)"))
2024-11-11 16:40:56,406 - INFO - JobSpy:Linkedin - finished scraping


TLSClientExeption: failed to do request: Get "https://www.glassdoor.com//Job/computer-science-jobs.htm": read tcp 10.8.16.39:52430->104.16.25.46:443: read: connection reset by peer